## 기간, 함수 설정


In [79]:
import pandas as pd
us1y_data = pd.read_csv('미국 국채 수익률 곡선.csv')
kr1y_data = pd.read_csv('국고채 만기별 수익률.csv')
us1y_data["수익률"] = (1+ 0.01*us1y_data["수익률"]) ** (1/365)
kr1y_data["수익률"] = (1+0.01*kr1y_data["수익률"]) ** (1/365)

us1y_data = us1y_data.set_index("날짜")
kr1y_data = kr1y_data.set_index("날짜")
us1y_data


,수익률
날짜,
1990-01-02,1.000206
1990-01-03,1.000207
1990-01-04,1.000206
1990-01-05,1.000206
1990-01-08,1.000206
...,...
2023-10-30,1.000144
2023-10-31,1.000145
2023-11-01,1.000143


In [78]:
1.004574**365

5.289392678515361

In [66]:
from datetime import datetime, timedelta
import yfinance as yf
import pandas as pd

# 기간 설정
start_date = "2018-10-31"
end_date = "2023-11-01"

# 원/달러 환율 데이터 가져오기


def get_exchange_rate(start_date, end_date):
    krw_usd = yf.Ticker("KRW=X")

    exchange_rate_data = krw_usd.history(
        period="1d", start=start_date, end=end_date)

    return exchange_rate_data

# 매수 기준 환율 설정


def get_standard(standard_date):
    YYYY, MM, DD = map(int, standard_date.split('-'))
    start_date = datetime(YYYY, MM, DD) - timedelta(days=30)
    end_date = datetime(YYYY, MM, DD)

    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")

    krw_usd = yf.Ticker("KRW=X")
    data = krw_usd.history(
        period="1d", start=start_date_str, end=end_date_str)

    return data['Close'].mean()


def get_dates(start_date_str, end_date_str):
    # 시작 날짜와 종료 날짜 설정
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")  # 날짜 문자열을 datetime 객체로 변환
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")  # 날짜 문자열을 datetime 객체로 변환

    # 날짜 범위 생성
    date_range = []
    current_date = start_date

    while current_date <= end_date:
        date_range.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
        
    return date_range

def make_df(start_date_str, end_date_str):
    dates = get_dates(start_date_str, end_date_str)

    df = pd.DataFrame()
    df["날짜"] = dates

    return df
    

In [70]:
df = make_df("2018-10-31","2023-11-01")
df

,날짜
0,2018-10-31
1,2018-11-01
2,2018-11-02
3,2018-11-03
4,2018-11-04
...,...
1823,2023-10-28
1824,2023-10-29
1825,2023-10-30
1826,2023-10-31


## 변수 설정


In [44]:
# 원/달러 환율 데이터, 기준환율 가져오기
exchange_rate_data = get_exchange_rate(start_date, end_date)
exchange_rate_data = exchange_rate_data.reset_index()

# 현재 매수 횟수
purchases_count = 0

# 최대 분할매수 횟수 (매수 구간 수)
max_purchases_count = 10

# 초기 매수기준환율, 초기 자본금(원)
standard_exchange_rate = 0
initial_capital = 1000000

# 초기 누적 수익
cumulative_profit = 0

# 보유 자본금
holding_won = initial_capital
holding_dollars = 0

# 거래 내용 저장
history = []
trading_count = 1
trading_history = pd.DataFrame(
    columns=['거래 #', '매수 날짜', '매수 환율', '매도 날짜', '매도 환율', '수익', '누적 수익'])

## 백테스팅


In [46]:
for i in range(len(exchange_rate_data)):
    date = str(str(exchange_rate_data.iloc[i]['Date'])[:10])
    current_exchange_rate = exchange_rate_data.iloc[i]['Open']

    holding_won *= kr1y_data.loc[date]
    holding_dollars *= us1y_data.loc[date]
    if purchases_count == 0:
        # 매수 기준 환율 설정
        standard_exchange_rate = get_standard(date) // 1
        # print('매수기준 환율:', standard_exchange_rate)
        # 추가 매수 간격 설정
        additional_purchase_interval = standard_exchange_rate * 0.005

    # print('\033[37m', "현재 환율:", current_exchange_rate)
    if 0 <= purchases_count < max_purchases_count:
        # 매수 조건
        if current_exchange_rate <= standard_exchange_rate:
            history.append([date, current_exchange_rate, holding_won //
                           (max_purchases_count - purchases_count)])
            holding_dollars += holding_won // (
                max_purchases_count - purchases_count) / current_exchange_rate
            holding_won -= holding_won // (max_purchases_count -
                                           purchases_count)
            standard_exchange_rate -= additional_purchase_interval
            purchases_count += 1
            # print('\033[32m', "매수:", current_exchange_rate)

    # 매도 조건
        elif current_exchange_rate > standard_exchange_rate + additional_purchase_interval and len(history) != 0:
            his = history.pop()
            profit = his[2] * ((current_exchange_rate / his[1]) - 1)
            cumulative_profit += profit
            new_trade = pd.DataFrame({'거래 #': [trading_count],
                                      '매수 날짜': [his[0]],
                                      '매수 환율': [his[1]],
                                      '매도 날짜': [date],
                                      '매도 환율': [current_exchange_rate],
                                      '수익': [str(round(profit, 5)) + ' KRW'],
                                      '누적 수익': [str(round(cumulative_profit, 5)) + ' KRW (' + str(round(cumulative_profit / initial_capital * 100, 1)) + '%)']})
            trading_history = pd.concat(
                [trading_history, new_trade], ignore_index=True)
            holding_won += holding_dollars // (purchases_count + 1) * \
                current_exchange_rate
            holding_dollars -= holding_dollars // (purchases_count + 1)
            standard_exchange_rate += additional_purchase_interval
            purchases_count -= 1
            trading_count += 1
            # print('\033[31m',"매도:", current_exchange_rate)
            # print('매수 환율 -> 매도 환율', history.pop(), '->', standard_exchange_rate)
    elif purchases_count > 0 and current_exchange_rate > standard_exchange_rate + additional_purchase_interval and len(history) != 0:
        his = history.pop()
        new_trade = pd.DataFrame({'거래 #': [trading_count],
                                  '매수 날짜': [his[0]],
                                  '매수 환율': [his[1]],
                                  '매도 날짜': [date],
                                  '매도 환율': [current_exchange_rate],
                                  '수익': [str(round(profit, 5)) + ' KRW'],
                                  '누적 수익': [str(round(cumulative_profit, 5)) + ' KRW (' + str(round(cumulative_profit / initial_capital * 100, 1)) + '%)']})
        trading_history = pd.concat(
            [trading_history, new_trade], ignore_index=True)
        holding_won += holding_dollars // (purchases_count + 1) * \
            current_exchange_rate
        holding_dollars -= holding_dollars // (purchases_count + 1)
        standard_exchange_rate += additional_purchase_interval
        purchases_count -= 1
        trading_count += 1
        # print('\033[31m',"매도:", current_exchange_rate)
        # print('매수 환율 -> 매도 환율', history.pop(), '->', standard_exchange_rate)

    # print(holding_won + (holding_dollars * current_exchange_rate))

KeyError: '2018-11-12'

## 거래목록


In [38]:
trading_history = trading_history.set_index('거래 #')
trading_history

,매수 날짜,매수 환율,매도 날짜,매도 환율,수익,누적 수익
거래 #,,,,,,
1,2018-11-06,1120.579956,2018-11-12,1130.880005,919.17125 KRW,919.17125 KRW (0.1%)
2,2018-11-05,1116.109985,2018-11-13,1138.250000,1983.67678 KRW,2902.84804 KRW (0.3%)
3,2018-11-16,1127.219971,2018-11-21,1129.349976,178.65118 KRW,3081.49922 KRW (0.3%)
4,2018-11-22,1126.800049,2018-11-23,1128.199951,115.63233 KRW,3197.13155 KRW (0.3%)
5,2018-11-02,1124.000000,2018-11-26,1133.310059,828.29703 KRW,4025.42857 KRW (0.4%)
...,...,...,...,...,...,...
150,2022-08-09,1299.030029,2022-08-10,1307.560059,653.76993 KRW,80582.63014 KRW (8.1%)
151,2022-08-11,1296.699951,2022-08-12,1306.829956,778.76124 KRW,81361.39139 KRW (8.1%)
152,2022-08-05,1302.079956,2022-08-16,1313.180054,848.75581 KRW,82210.14719 KRW (8.2%)


In [ ]:
for i in range(len(exchange_rate_data)):
    date = str(str(exchange_rate_data.iloc[i]['Date'])[:10])
    current_exchange_rate = exchange_rate_data.iloc[i]['Open']
    
    # 채권 수익률 반영   
    kor_ror, us_ror = get_ROR(date)
    holding_won *= kor_ror
    holding_dollars *= us_ror
    
    if purchases_count == 0:
        # 매수 기준 환율 설정
        standard_exchange_rate = get_standard(date) // 1
        # 추가 매수 간격 설정
        additional_purchase_interval = standard_exchange_rate * 0.005
        

    if 0 <= purchases_count <= max_purchases_count:
    # 매수 조건
        if current_exchange_rate <= standard_exchange_rate and purchases_count != 10:
            history.append([date, current_exchange_rate, holding_won // (max_purchases_count - purchases_count)])
            holding_dollars += holding_won // (max_purchases_count - purchases_count) / current_exchange_rate
            holding_won -= holding_won // (max_purchases_count - purchases_count)
            standard_exchange_rate -= additional_purchase_interval
            purchases_count += 1

                
    # 매도 조건
        elif (current_exchange_rate > standard_exchange_rate + additional_purchase_interval) and len(history) != 0:
            his = history.pop()
            profit = his[2] * ((current_exchange_rate / his[1]) - 1)
            cumulative_profit += profit
            
            ###### 거래 목록 최신화 #####
            new_trade = pd.DataFrame({'거래 #': [trading_count], 
                                    '매수 날짜': [his[0]], 
                                    '매수 환율': [his[1]], 
                                    '매도 날짜': [date], 
                                    '매도 환율': [current_exchange_rate], 
                                    '수익': [str(round(profit, 5)) + ' KRW'],
                                    '누적 수익': [str(round(cumulative_profit, 5)) + ' KRW (' + str(round(cumulative_profit / initial_capital * 100, 1)) + '%)']})
            trading_history = pd.concat([trading_history, new_trade], ignore_index=True)
            
            holding_won += holding_dollars // (purchases_count + 1) * current_exchange_rate
            holding_dollars -= holding_dollars // (purchases_count + 1)
            standard_exchange_rate += additional_purchase_interval
            purchases_count -= 1
            trading_count += 1
    # elif purchases_count == 10 and (current_exchange_rate > standard_exchange_rate + additional_purchase_interval) and len(history) != 0:
    #     his = history.pop()
    #     profit = his[2] * ((current_exchange_rate / his[1]) - 1)
    #     cumulative_profit += profit
        
    #     ###### 거래 목록 최신화 #####
    #     new_trade = pd.DataFrame({'거래 #': [trading_count], 
    #                             '매수 날짜': [his[0]], 
    #                             '매수 환율': [his[1]], 
    #                             '매도 날짜': [date], 
    #                             '매도 환율': [current_exchange_rate], 
    #                             '수익': [str(round(profit, 5)) + ' KRW'], 
    #                             '누적 수익': [str(round(cumulative_profit, 5)) + ' KRW (' + str(round(cumulative_profit / initial_capital * 100, 1)) + '%)']})
    #     trading_history = pd.concat([trading_history, new_trade], ignore_index=True)
        
    #     holding_won += holding_dollars // (purchases_count + 1) * current_exchange_rate
    #     holding_dollars -= holding_dollars // (purchases_count + 1)
    #     standard_exchange_rate += additional_purchase_interval
    #     purchases_count -= 1
    #     trading_count += 1
        
    print('수익률:', holding_won + holding_dollars * current_exchange_rate)
    print('날짜', date, 'purchases_count', purchases_count)